In [1]:
"""
    Script to stream news articles and compare sentiment against cryptocurrency price
    Author: Ross MacWilliam
    Date: 01/02/2021
"""

import re
import csv
from time import sleep
import requests
import json
import pandas as pd
import numpy as np
import requests
import bs4
import psycopg2
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, date

import pandas_datareader as pdr
import matplotlib.pyplot as plt

import shrimpy
import plotly.graph_objects as go
from newsapi import NewsApiClient
from yahoofinancials import YahooFinancials
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
class DatabaseClient:
    """
    Functionality for inserting and reading from the database
    """

    def __init__(self):
        # Connect to db
        self.con = psycopg2.connect(
            host='DESKTOP-VSKLGP8',
            database='postgres',
            user='postgres',
            password='postgres'
        )
        self.article_table = pd.DataFrame()

    def read_db(self):
        cur = self.con.cursor()

        cur.execute("SELECT full_date, hour, trading_symbol, comp_sentiment FROM sentiment_dim"
                    " JOIN date_dim ON sentiment_dim.dateid = date_dim.dateid"
                    " JOIN time_dim ON sentiment_dim.timeid = time_dim.timeid;")

        rows = cur.fetchall()

        self.article_table = pd.DataFrame(data=rows, columns=['date', 'hour', 'trading_symbol', 'compound'])


        return self.article_table

    def insert_db(self, df):
        cur = self.con.cursor()

        for i in range(len(df)):
            cur.execute(f"SELECT dateid FROM date_dim WHERE full_date = '{df.dateid[i]}'")
            dateid = cur.fetchall()

            cur.execute(f"SELECT timeid FROM time_dim WHERE hour = '{df.timeid[i]}'")
            timeid = cur.fetchall()
            
            cur.execute(
                "INSERT INTO ma_sentiment_dim (dateid, timeid, trading_symbol, comp_sentiment, sma, ema)"
                f" VALUES ({dateid[0][0]}, {timeid[0][0]}, '{df['trading_symbol'][i]}', {df['compound'][i]}, {df['SMA'][i]}, {df['EWM'][i]});")

        self.con.commit()

        self.con.close()


In [3]:
class SentimentProcessor:
    """
    Functionality for inserting and reading from the database
    """

    def __init__(self):
        pass
    
    def calculate_moving_averages(self, overall_headlines):
        overall_headlines['datetime'] = pd.to_datetime(df.date, infer_datetime_format=True) + df.hour.astype('timedelta64[h]')

        overall_headlines.index =  pd.DatetimeIndex(overall_headlines['datetime'])
        overall_headlines = overall_headlines.drop(['datetime'], axis=1)

        # Remove duplicate columns
        overall_headlines = overall_headlines[~overall_headlines.index.duplicated()]

        overall_headlines = overall_headlines.asfreq('H')
        
        overall_headlines['SMA'] = overall_headlines.compound.rolling(window=48,min_periods=1).mean()
        overall_headlines['EWM'] = overall_headlines.compound.ewm(span=14).mean()
        overall_headlines['trading_symbol'] = 'BTC'
        overall_headlines['timeid'] = overall_headlines.index.hour
        overall_headlines['dateid'] = overall_headlines.index.strftime('%d/%m/%Y')
        
        overall_headlines.loc[(overall_headlines['compound'].isnull()), 'compound'] = 'NULL'
        overall_headlines.loc[(overall_headlines['SMA'].isnull()), 'SMA'] = 'NULL'
        overall_headlines.loc[(overall_headlines['EWM'].isnull()), 'EWM'] = 'NULL'
        
        return overall_headlines

In [4]:
database_client = DatabaseClient()
df = database_client.read_db()

sentiment_processor = SentimentProcessor()
overall_headlines = sentiment_processor.calculate_moving_averages(df)

database_client.insert_db(overall_headlines)